<h1 align = center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

## Introduction
This project is designed for clustering neighborhoods in Toronto by K-Means.

## Table of Contents

- Webscrap and Wrangle Data
- Explore Neighborhood in Toronto
- Analyze Each Neighborhood
- Cluster Neighborhoods
- Examine Clusters

#### Use BeautifulSoup to webscrap the neighborhood data in Toronto

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import requests
import geopy

In [3]:
#Firsly webcrap the neighborhoods of Toronto from Wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url)

In [4]:
soup = BeautifulSoup(html.text,'lxml')

In [5]:
table_contents = []
table = soup.find('table')

In [6]:
for row in table.find_all('td'):
    cell = {}
    if row.span.text == 'Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

In [7]:
df = pd.DataFrame(table_contents)

In [8]:
df['Borough'] = df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [9]:
df.shape

(103, 3)

#### Use geocoder library to get the latitude and longitude value of Toronto.

In [10]:
postal_code = list(df['PostalCode'])

In [11]:
geolocator = Nominatim(user_agent = 'Toronto')
location = geolocator.geocode('Toronto')
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Toronto are {}, {}'.format(latitude,longitude))

The geographical coordinate of Toronto are 43.6534817, -79.3839347


In [12]:
g = geolocator.geocode('{}, Toronto, Ontario'.format('M4A'))

In [13]:
#geolocator = Nominatim(user_agent = 'Toronto')
#for p in postal_code:
#   g = None
#    time = 0
#    while(g is None):
#        g = geolocator.geocode('{}, Toronto, Ontario'.format(p))
#        print('search {} for {} times'.format(p,time+1))
#    lat_long = [g.latitude,g.longitude]
#    latitude.append(lat_long[0])
#    longitude.append(lat_long[1])
#    print('{} done'.format(p))

In [14]:
geoco = pd.read_csv('Geospatial_Coordinates.csv')

In [15]:
df = df.merge(geoco,how = 'left',left_on = 'PostalCode',right_on = 'Postal Code')

In [16]:
df = df.drop('Postal Code',axis=1)

In [17]:
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


#### Create a map of Toronto with neighborhoods superimposed on top

In [18]:
address = 'Toronto'
geolocator = Nominatim(user_agent = 'Toronto')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [19]:
map_toronto = folium.Map(location = [latitude,longitude],zoom_start=10)
for lat,lon,borough,nei in zip(df['Latitude'],df['Longitude'],df['Borough'],df['Neighborhood']):
    label = '{}, {}'.format(nei,borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker([lat,lon],radius = 5,popup=label,color = 'blue',fill = True,fill_color = '#3186cc',fill_opacity = 0.7,parse_html = False).add_to(map_toronto)

In [20]:
map_toronto

#### Define Foursquare Credentials and Version

In [21]:
CLIENT_ID = 'WTXPT3Y1TAJEVADM1R4CDTFDH3BAFXUDFGNBABSWEDLEVEFV' # your Foursquare ID
CLIENT_SECRET = 'V52T4ELLOUD50L0RHXHB5W3MHBUFNQUFTIGDWJD4A22Y5EEZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WTXPT3Y1TAJEVADM1R4CDTFDH3BAFXUDFGNBABSWEDLEVEFV
CLIENT_SECRET:V52T4ELLOUD50L0RHXHB5W3MHBUFNQUFTIGDWJD4A22Y5EEZ


In [24]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
toronto_venues = getNearbyVenues(names = df['Neighborhood'],latitudes = df['Latitude'],longitudes = df['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

In [58]:
toronto_venues.to_csv('Toronto Venues.csv')

#### Explore neighborhoods

In [157]:
missed_neighborhoods = [i for i in list(df['Neighborhood']) if i not in list(toronto_venues['Neighborhood'])]

In [159]:
print('There are four neighboods that we could not get venues from Foursquare: ',missed_neighborhoods)

There are four neighboods that we could not get venues from Foursquare:  ['Islington Avenue', 'York Mills, Silver Hills', 'Willowdale, Newtonbrook', 'Upper Rouge']


In [160]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",23,23,23,23,23,23
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
...,...,...,...,...,...,...
Willowdale South,35,35,35,35,35,35
Willowdale West,7,7,7,7,7,7
Woburn,3,3,3,3,3,3


In [161]:
print('There are {} unique categories'.format(len(toronto_venues['Venue Category'].unique())))

There are 272 unique categories


#### Analyze Each Neighborhood

In [162]:
columns = []
toronto_getdummies = pd.get_dummies(toronto_venues[['Venue Category']],
                                   prefix = '',prefix_sep='')
columns = list(toronto_getdummies.columns)

In [163]:
columns.remove('Neighborhood')
columns =['Neighborhood']+list(columns)

In [164]:
toronto_getdummies = toronto_getdummies[columns]

In [165]:
toronto_getdummies['Neighborhood'] = toronto_venues['Neighborhood']

In [166]:
toronto_groupby = toronto_getdummies.groupby('Neighborhood').mean().reset_index()

In [167]:
toronto_groupby

,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.04,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,Willowdale South,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.000000,0.028571,0.0,0.0,0.0,0.0,0.0
95,Willowdale West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
96,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
97,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.142857,0.000000,0.0,0.0,0.0,0.0,0.0


#### Find the top 5 venues in each neighborhood

In [168]:
toronto_groupby[toronto_groupby['Neighborhood'] == 'Woburn'].T.reset_index()

,index,96
0,Neighborhood,Woburn
1,Accessories Store,0
2,Adult Boutique,0
3,Airport,0
4,Airport Food Court,0
...,...,...
267,Warehouse Store,0
268,Wine Bar,0
269,Wings Joint,0
270,Women's Store,0


In [169]:
num_top_venues = 5
for hood in toronto_groupby['Neighborhood']:
    print('----'+hood+'----')
    temp = toronto_groupby[toronto_groupby['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp =temp.iloc[1:]
    temp['freq'] = temp['freq'].astype('float')
    temp =  temp.round({'freq':2})
    print(temp.sort_values('freq', ascending =  False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0                     Lounge  0.25
1             Breakfast Spot  0.25
2  Latin American Restaurant  0.25
3               Skating Rink  0.25
4          Accessories Store  0.00


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.29
1     Coffee Shop  0.14
2  Sandwich Place  0.14
3             Pub  0.14
4        Pharmacy  0.14


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                       Bank  0.09
1                Coffee Shop  0.09
2                       Park  0.04
3               Intersection  0.04
4  Middle Eastern Restaurant  0.04


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4        Movie Theater  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0         Pizza Place  0.08
1          Restaurant  0.08

In [170]:
def return_most_common_venues(row,num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending = False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [180]:
num_top_venues = 10
indicators = ['st','nd','rd']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
neighborhoods_venues_sorted = pd.DataFrame(columns = columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_groupby['Neighborhood']
for ind in np.arange(toronto_groupby.shape[0]):
    neighborhoods_venues_sorted.iloc[ind,1:] = return_most_common_venues(toronto_groupby.iloc[ind,:],num_top_venues)
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Skating Rink,Breakfast Spot,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
1,"Alderwood, Long Branch",Pizza Place,Pharmacy,Sandwich Place,Coffee Shop,Pub,Gym,Airport Lounge,Department Store,Ethiopian Restaurant,Escape Room
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Frozen Yogurt Shop,Bridal Shop,Sandwich Place,Diner,Deli / Bodega,Restaurant,Intersection,Supermarket
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Restaurant,Pizza Place,Hobby Shop,Juice Bar,Fast Food Restaurant,Butcher,Indian Restaurant


In [182]:
neighborhoods_venues_sorted.shape

(99, 11)

#### Custer Neighborhoods

In [183]:
Kclusters = 5
toronto_cluster = toronto_groupby.drop('Neighborhood',axis=1)

In [184]:
kms = KMeans(n_clusters = Kclusters, random_state = 0).fit(toronto_cluster)

In [185]:
kms.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       3, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 3, 0, 0, 0,
       1, 0, 0, 1, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 1], dtype=int32)

In [186]:
neighborhoods_venues_sorted.insert(0, 'Cluster Lables',kms.labels_)

In [187]:
neighborhoods = df.join(neighborhoods_venues_sorted.set_index('Neighborhood'),on = 'Neighborhood',how = 'right')

In [193]:
neighborhoods.reset_index().drop('index',axis=1)

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Lables,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1S,Scarborough,Agincourt,43.794200,-79.262029,0,Lounge,Latin American Restaurant,Skating Rink,Breakfast Spot,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
1,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484,0,Pizza Place,Pharmacy,Sandwich Place,Coffee Shop,Pub,Gym,Airport Lounge,Department Store,Ethiopian Restaurant,Escape Room
2,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,0,Coffee Shop,Bank,Frozen Yogurt Shop,Bridal Shop,Sandwich Place,Diner,Deli / Bodega,Restaurant,Intersection,Supermarket
3,M2K,North York,Bayview Village,43.786947,-79.385975,0,Café,Bank,Japanese Restaurant,Chinese Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,0,Coffee Shop,Sandwich Place,Italian Restaurant,Restaurant,Pizza Place,Hobby Shop,Juice Bar,Fast Food Restaurant,Butcher,Indian Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,M2N,North York,Willowdale South,43.770120,-79.408493,0,Ramen Restaurant,Coffee Shop,Shopping Mall,Restaurant,Café,Sushi Restaurant,Pizza Place,Bubble Tea Shop,Grocery Store,Discount Store
95,M2R,North York,Willowdale West,43.782736,-79.442259,0,Pizza Place,Supermarket,Discount Store,Pharmacy,Coffee Shop,Butcher,Grocery Store,Airport Service,Falafel Restaurant,Ethiopian Restaurant
96,M1G,Scarborough,Woburn,43.770992,-79.216917,2,Coffee Shop,Korean BBQ Restaurant,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
97,M4C,East York,Woodbine Heights,43.695344,-79.318389,0,Park,Athletics & Sports,Beer Store,Skating Rink,Video Store,Intersection,Curling Ice,Dog Run,Diner,Discount Store


#### Create a map for the clusters

In [195]:
map_clusters = folium.Map(location = [latitude,longitude], zoom_start = 10)
x = np.arange(Kclusters)
ys = [i + x + (i*x)**2 for i in range(Kclusters)]
colors_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [196]:
neighborhoods['Cluster Lables'] = neighborhoods['Cluster Lables'].astype('int')

In [197]:
markers_colors = []
for lat,long,poi,cluster in zip(neighborhoods['Latitude'],neighborhoods['Longitude'],
                                neighborhoods['Neighborhood'],neighborhoods['Cluster Lables']):
    label = folium.Popup(str(poi) + 'Cluster' +str(cluster),parse_html = True)
    folium.CircleMarker(
    [lat,long],
    radius = 5,
    popup = label,
    color = rainbow[int(cluster)-1],
    fill = True,
    fill_color = rainbow[int(cluster)-1],
    fill_opacity = 0.7).add_to(map_clusters)
map_clusters

##### cluster1

In [198]:
neighborhoods[neighborhoods['Cluster Lables'] == 0]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Lables,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
78,M1S,Scarborough,Agincourt,43.794200,-79.262029,0,Lounge,Latin American Restaurant,Skating Rink,Breakfast Spot,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
93,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484,0,Pizza Place,Pharmacy,Sandwich Place,Coffee Shop,Pub,Gym,Airport Lounge,Department Store,Ethiopian Restaurant,Escape Room
28,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,0,Coffee Shop,Bank,Frozen Yogurt Shop,Bridal Shop,Sandwich Place,Diner,Deli / Bodega,Restaurant,Intersection,Supermarket
39,M2K,North York,Bayview Village,43.786947,-79.385975,0,Café,Bank,Japanese Restaurant,Chinese Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
55,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,0,Coffee Shop,Sandwich Place,Italian Restaurant,Restaurant,Pizza Place,Hobby Shop,Juice Bar,Fast Food Restaurant,Butcher,Indian Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,M9P,Etobicoke,Westmount,43.696319,-79.532242,0,Pizza Place,Chinese Restaurant,Intersection,Sandwich Place,Playground,Coffee Shop,Middle Eastern Restaurant,Discount Store,Yoga Studio,Dim Sum Restaurant
71,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849,0,Accessories Store,Vietnamese Restaurant,Middle Eastern Restaurant,Sandwich Place,Bakery,Auto Garage,Concert Hall,Construction & Landscaping,Escape Room,Electronics Store
59,M2N,North York,Willowdale South,43.770120,-79.408493,0,Ramen Restaurant,Coffee Shop,Shopping Mall,Restaurant,Café,Sushi Restaurant,Pizza Place,Bubble Tea Shop,Grocery Store,Discount Store
72,M2R,North York,Willowdale West,43.782736,-79.442259,0,Pizza Place,Supermarket,Discount Store,Pharmacy,Coffee Shop,Butcher,Grocery Store,Airport Service,Falafel Restaurant,Ethiopian Restaurant


In [200]:
neighborhoods[neighborhoods['Cluster Lables'] == 1]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Lables,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512,1,Park,Women's Store,Pool,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
40,M3K,North York,Downsview East,43.737473,-79.464763,1,Airport,Park,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
68,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307,1,Jewelry Store,Park,Sushi Restaurant,Trail,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
10,M6B,North York,Glencairn,43.709577,-79.445073,1,Pizza Place,Park,Bakery,Japanese Restaurant,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
77,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724,1,Park,Sandwich Place,Bus Line,Mobile Phone Shop,Yoga Studio,Dog Run,Diner,Discount Store,Distribution Center,Doner Restaurant
61,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Bus Line,Park,Dim Sum Restaurant,Swim School,Yoga Studio,Dog Run,Diner,Discount Store,Distribution Center,Doner Restaurant
85,M1V,Scarborough,"Milliken, Agincourt North, Steeles East, L'Amo...",43.815252,-79.284577,1,Intersection,Playground,Park,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
83,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,1,Restaurant,Park,Trail,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
49,M6L,North York,"North Park, Maple Leaf Park, Upwood Park",43.713756,-79.490074,1,Basketball Court,Park,Bakery,Construction & Landscaping,Yoga Studio,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
0,M3A,North York,Parkwoods,43.753259,-79.329656,1,Park,Construction & Landscaping,Food & Drink Shop,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run


In [201]:
neighborhoods[neighborhoods['Cluster Lables'] == 2]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Lables,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,M1G,Scarborough,Woburn,43.770992,-79.216917,2,Coffee Shop,Korean BBQ Restaurant,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio


In [202]:
neighborhoods[neighborhoods['Cluster Lables'] == 3]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Lables,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,M9M,North York,"Humberlea, Emery",43.724766,-79.532242,3,Baseball Field,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Falafel Restaurant
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,3,Pool,Baseball Field,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
